# GNIP Historical PowerTrack 2.0 API

In [14]:
JOB_DIR    = './gnip-geo-query'
JOB_CONFIG = './gnip-geo-query/geo_query_job.yml'

In [6]:
import json, yaml, requests, sys, os
from requests.auth import HTTPBasicAuth

In [7]:
ENDPOINT = 'https://gnip-api.gnip.com/historical/powertrack/accounts/CUResearch/publishers/twitter/jobs.json'
BASE_URI = 'https://gnip-api.gnip.com/historical/powertrack/accounts/CUResearch/publishers/twitter/jobs/'

class HistoricalPowerTrackJob():
    """http://support.gnip.com/apis/historical_api2.0/api_reference.html#Create"""
    def __init__(self,config):
        self.config = yaml.load(open(config,'r'))

        #Create authentication
        auth_file = yaml.load(open('/home/anderstj/credentials/gnip.yaml'))
        self.auth=HTTPBasicAuth(auth_file['username'],auth_file['password'])
    def load_rules(self):
        try:
            rules = json.load(open(JOB_DIR+"/"+self.config['rules_file'],'r'))
        except:
            print("ERROR parsing rules")
            sys.exit(1)
        return rules
    
    def make_payload(self):
        self.payload = {
            'title':self.config['title'],
            'fromDate':self.config['from_date'],
            'toDate':self.config['to_date'],
            'publisher':'twitter',
            'streamType':'track_v2',
            'dataFormat':'activity_streams',
            'rules': self.load_rules()
        }
    
    def testAPI(self):
        response = requests.get(ENDPOINT,auth=self.auth)
        return response.json()
    
    def status():
        response = requests.get(BASE_URI+self.uuid+".json", auth=self.auth)
        return response.json()
    
    def submit(self):
#         response = #Post the job
        self.uuid = #Get the uuid from the response...
        return False
    
    def reject(self):
        response
        return False
    
    def accept(self):
        return False
        

In [9]:
job = HistoricalPowerTrackJob(JOB_CONFIG)
job.make_payload()
job.payload

{'dataFormat': 'activity_streams',
 'fromDate': 201608010000,
 'publisher': 'twitter',
 'rules': [{'value': 'cow'}, {'tag': 'pets', 'value': 'dog'}],
 'streamType': 'track_v2',
 'title': 'HurricaneMatthewGeoQuery_v1',
 'toDate': 201610270000}

## Test Credentials, GET Request
A get request to this endpoint with the proper authentication returns a list of our current jobs.

In [169]:
job.testAPI()

{'delivered': {'activityCount': 294709,
  'jobCount': 10,
  'jobDaysRun': 384,
  'period': 'trial',
  'since': '2016-08-31T15:49:34Z'},
 'jobs': [{'expiresAt': '2016-10-23T09:25:10Z',
   'fromDate': '201608010000',
   'jobURL': 'https://gnip-api.gnip.com:443/historical/powertrack/accounts/CUResearch/publishers/twitter/jobs/pz2kwntec9.json',
   'percentComplete': 0,
   'publisher': 'twitter',
   'status': 'rejected',
   'streamType': 'track_v2',
   'title': 'rb_job_test',
   'toDate': '201608012000',
   'uuid': 'pz2kwntec9'},
  {'expiresAt': '2016-10-31T22:46:07Z',
   'fromDate': '201608010000',
   'jobURL': 'https://gnip-api.gnip.com:443/historical/powertrack/accounts/CUResearch/publishers/twitter/jobs/vttpj4nx3y.json',
   'percentComplete': 100,
   'publisher': 'twitter',
   'status': 'delivered',
   'streamType': 'track_v2',
   'title': 'ruby_job_test',
   'toDate': '201608012000',
   'uuid': 'vttpj4nx3y'},
  {'expiresAt': '2016-10-24T19:29:21Z',
   'fromDate': '201109060000',
   'jo

# Historical Powertrack Request

## Part 1: Submit the job

In [170]:
#part1 = requests.post(uri,auth=auth)

## Final Part: Retrieve the Results
Check the status of the job until the results.json file is ready and then save it.

In [15]:
# os.mkdir(JOB_DIR+"/results")
job.status()